In [2]:
import pandas as pd
import numpy as np
import time
import string
from unidecode import unidecode

In [3]:
df_lyrics = pd.read_csv("data/lyrics.csv", sep =';')

df_lyrics[:5]

,ALink,SName,SLink,Lyric,ParentGenre,Genres,Genres_API
0,/andre-drake/,Bae ft. V.P Rap,/andre-drake/bae-ft-v-p-rap.html,[André Drake]. Minha princesa eu troco tudo pr...,Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
1,/andre-drake/,By Chance (Brazilian Remix),/andre-drake/by-chance-brazilian-remix.html,"[Refrão]. Mais views, pode ser?. Cê sabe, que ...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
2,/andre-drake/,Casa do Terror,/andre-drake/casa-do-terror.html,"4 Paredes uma jaula, na minha mão uma faca. Um...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
3,/andre-drake/,Na Madrugada,/andre-drake/na-madrugada.html,"[Refrão 1]. Na ""madruga"" não consigo dormir. m...",Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"
4,/andre-drake/,V.P Convida Parte 2,/andre-drake/v-p-convida-parte-2.html,[Vp Rap]. Vp de novo que convida. Dessa vez se...,Black Music,"Black Music, Hip Hop, Rap","Black Music, Hip Hop, Rap"


In [6]:
# Filtrando dataset de músicas para deixar apenas gêneros com mais de 2500 letras musicais.
serie_parent_genre = df_lyrics["ParentGenre"].value_counts()
generos_selecionados = list(serie_parent_genre[serie_parent_genre >2500].index)
df_lyrics = df_lyrics[df_lyrics["ParentGenre"].isin(generos_selecionados)]

In [7]:
# Removendo músicas instrumentais
df_lyrics = df_lyrics[ ~((df_lyrics.Lyric.str.contains("Instrumental")) & \
                         (df_lyrics.Lyric.str.split(' ').map(lambda x: len(x)<20))) ]

In [8]:
# limit é uma variável para determinar quantas letras devem ser utilizadas para o processamento
# útil apenas para debug, caso contrário utilizar o tamanho total do dataset
limit = df_lyrics["Lyric"].shape[0]
serie_lyrics = df_lyrics["Lyric"][0:limit]

In [9]:
# Removendo conteúdo [] e ()
# Transformando strings em minúsculo
# Removendo pontuação e acentuação

def process_lyric(lyric):
    try:
        lyric = unidecode(lyric)
        lyric = lyric.translate(str.maketrans('', '', string.punctuation))
    except:
        pass
    return lyric

start = time.time()  

serie_lyrics = serie_lyrics.str.replace("[\(\[].*?[\)\]]","")
serie_lyrics = serie_lyrics.str.lower()
serie_lyrics = serie_lyrics.map(process_lyric)

end = time.time()  
print('\nTime(s): '+str("%.2f" % (end - start)+'s\n'))


Time(s): 34.94s



In [10]:
# Criando e salvando train_data e test_data. (Proporção 80/20) 
# Tais arquivos serão utilizados pelo pytext para treino e avaliação do resultado.

df_all = pd.DataFrame({"Genre": df_lyrics["ParentGenre"][0:limit], "Lyric": serie_lyrics})

df_all["Genre"] = df_all["Genre"].str.replace(" ","_")

df_all = df_all[df_all['Lyric'].str.len()>0]

msk = np.random.rand(len(df_all)) < 0.8

df_train = df_all[msk]
df_test  = df_all[~msk]

df_train.to_csv('train_data.tsv', sep='\t', index = False, header = False)
df_test.to_csv('test_data.tsv', sep='\t' , index = False, header = False)